In [ ]:
import pandas as pd
import numpy as np
import pydot as pydot
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import plotly.express as px
plt.style.use("ggplot")
rcParams['figure.figsize'] = (12,6)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn import metrics
from sklearn import tree
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow import keras
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn import over_sampling
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import initializers
#from hyperas.distributions import choice, uniform

In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 16.9 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root_path = '/content/drive/My Drive/'

Mounted at /content/drive


In [ ]:
X_train = pd.read_csv(root_path+'X_train_lvl1.csv')
y_train = pd.read_csv(root_path+'y_train_lvl1.csv')
X_test = pd.read_csv(root_path+'X_test_lvl1.csv')
y_test = pd.read_csv(root_path+'y_test_lvl1.csv')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = ~y_train+2
y_test = ~y_test+2

X_train = np.array(X_train)
X_test = np.array(X_test)

scaler = preprocessing.MinMaxScaler((0, 1)) # normalize data
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_scaled = np.clip(X_scaled, -5, 5)
X_test_scaled = np.clip(X_test_scaled, -5, 5)

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_scaled, y_train)

print('Training labels shape:', y_resampled.shape)

print('Training features shape:', X_resampled.shape)

Hypertune NN

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  hp_units_1 = hp.Int('units1', min_value=16, max_value=512, step=48) # parameters for first layer
  hp_units_2 = hp.Int('units2', min_value=16, max_value=256, step=48) # parameters for second layer
  # Tune the number of units in the first Dense layer
  model.add(keras.layers.Dense(units=hp_units_1, input_shape = (X_train.shape[-1],), activation='relu', name='dense_1'))
  # Add next layers
  model.add(keras.layers.Dropout(0.2))
  # Tune the number of units in the second Dense layer
  model.add(keras.layers.Dense(units=hp_units_2, activation='relu', name='dense_2'))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  # Tune learning rate for the optimizer
  # Choose optimal value from 0.001, 0.0001, or 0.00001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss=keras.losses.BinaryCrossentropy(),
                  metrics = [keras.metrics.AUC(name='auc'), keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn')])


  return model

In [ ]:
# Define tuner
tuner = kt.Hyperband(
    hypermodel=model_builder,
    objective=kt.Objective('val_loss', direction="min"),
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='my_project'
)


In [ ]:
# Define early stopping
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# Perform hypertuning
tuner.search(X_resampled, y_resampled, epochs=10, validation_data = (X_test_scaled, np.ravel(y_test)), callbacks=[stop_early])
best_hp=tuner.get_best_hyperparameters()[0]